In [ ]:
"""
The model is designed to analyze the consensus sequences in DNA sequences

The input of this model will be in the 2D matrix:    
    dimension: n x 1000, n is the amount of the input samples

The output of the model will be in the 3D matrix:
    dimension: n x 1000 x 4, n is the amoutn of the input samples
               the meaning of the output is the position-wise appearance for each alphabets(ATCG)

"""

In [1]:
import os
import torch
import numpy as np
import math
import json
import time
import random
from collections import Counter, defaultdict
from sklearn.preprocessing import normalize
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer,\
    TransformerDecoder, TransformerDecoderLayer
from torch.nn.functional import softmax
from itertools import compress

In [2]:
'''
Global variables
'''

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

alphabet_dict = {'A' : 1, 'T' : 2, 'C' : 3, 'G' : 4}

combi_letters = {'A' : 1, 'T' : 2, 'C' : 3, 'G' : 4, 
                 'AT' : 5, 'AC': 6, 'AG' : 7, 'TC' : 8, 'TG' : 9, 
                 'CG' : 10, 'ATC' : 11, 'ATG' : 12, 'ACG' : 13,
                 'TCG' : 14, 'ATCG' : 15}
val_to_char = {1 : 'A', 2 : 'T', 3 : 'C', 4 : 'G'}

In [ ]:
# read the fasta files and make the list of DNA sequences
def read_file(path):
    f = open('../test.fa','r')
    seq = {}

    for line in f:
        if line.startswith('>'):
            name = line.replace('>', '').split()[0]
            seq[name] = ''
        else:
            seq[name] += line.replace('\n','').strip()

    f.close()

    seq_ls = []
    for name in seq:
        seq_ls.append(seq[name])

In [3]:
d_model = 512
dropout = 0.1
max_len = 500
nhead = 8
encoder_layer_nums = 6
decoder_layer_nums = 6
dim_ff = 512
# token: ATCG(1~4), <BOS or SOS>(0), <EOS>(16), <PAD>(17)
    # combinations: AT, AC, AG, TC, TG, CG, ATC, ATG, ACG, TCG, ATCG(5~15, total 11)
ntoken = 18

In [63]:
class TransformerModel(nn.Module): #done
    def __init__(
        self,
        d_model,
        dropout,
        max_len,
        nhead,
        encoder_layer_nums,
        decoder_layer_nums,
        dim_ff,
        ntoken,
    ):
        '''
        
        '''
        super(TransformerModel, self).__init__()
        
        self.d_model = d_model
        self.pos_encode = PositionalEmbedding(d_model, dropout, max_len)
        encoder_layer = TransformerEncoderLayer(d_model, nhead, dim_ff, dropout, batch_first = True)
        self.encoder = TransformerEncoder(encoder_layer, encoder_layer_nums)
        decoder_layer = TransformerDecoderLayer(d_model, nhead, dim_ff, dropout, batch_first = True)
        self.decoder = TransformerDecoder(decoder_layer, decoder_layer_nums)
        self.embedding = nn.Embedding(ntoken, d_model)
        self.out_embed = nn.Embedding(ntoken, d_model)
        self.output_linear = nn.Linear(d_model, ntoken)
        self.output_softmax = nn.Softmax(dim = -1)
        
        self.init_weights()

    
    def init_weights(self):
        initrange = 0.1
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.output_linear.bias.data.zero_()
        self.output_linear.weight.data.uniform_(-initrange, initrange)
    
    def forward(
        self,
        src,
        tgt,
        src_mask = None,
        tgt_mask = None,
        memory_mask = None,
        src_key_padding_mask = None,
        tgt_key_padding_mask = None,
        memory_key_padding_mask = None,
    ):
        
        src = self.embedding(src)
        tgt = self.embedding(tgt)
        
        src = self.pos_encode(src)
        tgt = self.pos_encode(tgt)
        
        memory = self.encoder(src, mask = src_mask, src_key_padding_mask = src_key_padding_mask)
        output = self.decoder(tgt, memory, 
                              tgt_mask = tgt_mask, 
                              memory_mask = memory_mask, 
                              tgt_key_padding_mask = tgt_key_padding_mask,
                              memory_key_padding_mask = memory_key_padding_mask
                             )
        output = self.output_linear(output)
        output = self.output_softmax(output)
        
        return output

In [64]:
class PositionalEmbedding(nn.Module): #done
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 1000):
        super(PositionalEmbedding, self).__init__()
        self.dropout = nn.Dropout(p = dropout)
        
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp( torch.arange(0, d_model, 2) * (-math.log(10000) / d_model) )
        pe = torch.zeros(1, max_len, d_model)
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
    
    def forward(self, x: Tensor):
        x = x + self.pe[0, :x.size(1), :].requires_grad_(False)
        output = self.dropout(x)
        return output
    

In [31]:
class MEME(object):
    def __init__(self, source_data, W, beta = 0.1):
        '''
        source_data will be list of strings
        '''
        
        self.source_data = source_data
        self.W = W
        self.L = 4
        self.beta = beta
        
        # fixed variables
        self.N = None
        self.l = None
        self.X = None
        self.n = None
        self.I = None
        self.letter_counts = None
        self.beta_i = np.zeros((1,self.L))
        
        # mutable variables, i.e. the value will be changed
        self.p_X1 = None
        self.p_X2 = None
        self.Z = None
        self.z = None
        self.f_ij = None
        self.e_ij = None
        self.lambda1 = 0
        self.lambda2 = 0
        
        self.init_variables()
        
        
    def init_variables(self):
        '''
        initialize the followings: X, f_i, z, I, lambda, N, small L, 
        '''
        
        # record N and small L, and also init beta_i
        self.N = len(self.source_data)
        self.l = list()
        for seq in self.source_data:
            self.l.append(len(seq))
            for s in seq:
                self.beta_i[0][alphabet_dict[s] - 1] += 1
        
        # trans beta_i as frequency and multi with customed beta
        self.beta_i /= np.sum(self.l)
        self.beta_i *= self.beta
                
        # init small z
        self.z = np.ones((self.N, max(self.l)))
        for idx in range(self.N):
            self.z[idx][(self.l[idx]- self.W + 1):] = 0
        self.e_ij = self.z.copy()
        
        # init W-mer set X
        # X will be the list of strings
        X_ls = list()
        for i in range(self.N):
            X_ls += [ self.source_data[i][j:j+self.W] for j in range( self.l[i] - self.W + 1 ) ]
        self.X = X_ls
        self.n = len(self.X)
        
        # init indicator
        self.I = self.indicator_function()
        
        # init lambda
        lamb_range_min = min( np.sqrt(self.N)/self.n, 1/(2*self.W) )
        lamb_range_max = max( np.sqrt(self.N)/self.n, 1/(2*self.W) )
        self.lambda1 = np.random.uniform(lamb_range_min, lamb_range_max)
        self.lambda2 = 1 - self.lambda1
        
        # init f_ij, size: ( 1 + W ) x L
        self.f_ij = np.zeros( ( (self.W + 1), self.L ) )
        
        # init letter_counts
        self.letter_counts = self.count_letter_appearance()
        
        # f_0 part
        count_num = defaultdict(int)
        for subseq in self.X:
            for s in subseq:
                count_num[s.upper()] += 1
        total = np.concatenate(self.letter_counts).sum()
        for letter in alphabet_dict:
            self.f_ij[0][alphabet_dict[letter] - 1] = count_num[letter]/total
        
        # f_i parts
        for pos in range(self.W):
            C = Counter(self.I.transpose()[pos])
            length = len(self.I)
            for val in range(4):
                self.f_ij[pos + 1][val] = C[val + 1] / length
        
        # conditional probabilities p_X1, p_X2
        
        self.p_X1, self.p_X2 = self.condi_distribution(self.f_ij)
        
        return
        
    def indicator_function(self): #done
        '''
        In article, it is the I(k,a) function for eq(7), (8)
        There will transfer the alphabets to the index
        Thus, the results look like 
        [
        [1,2,3,4],
        [4,3,2,1]
        ]

        return
        indicator: transfer input string into tensor indicator, size is n x W
        '''
        assert isinstance(self.X, list), 'Type of X is not list'
        indicator = list()
        for seq in self.X:
            indicator.append(list(map(lambda x: alphabet_dict[x], seq)))

        return np.array(indicator, dtype = 'int')
        
    
    def condi_distribution(self, freq_letter):
        '''
        Calculate the conditional distribution p(Xi | theta_j)
        eq(7),(8) in the MEME article
        To avoid the computation error for the digits, it will use ln() to make it being summation
    
        Arguments:
    
        freq_letter: the frequences for each letter in each position, size: (W + 1) x L
                     background ( 1 x L ) + motif ( W x L )
                     dtype: np.array
        ====================================================
        return: it will be the log form output
    
        p_Xi_1: conditional distribution of motif sequence, size: n x 1
        p_Xi_2: conditional distribution of background, size: n x 1
        '''
    
        p_Xi_1 = np.zeros(self.n)
        p_Xi_2 = np.zeros(self.n)
        f_0 = freq_letter[0]
        f_j = freq_letter[1:]
    
        for subseq in range(self.n):
            for pos in range(self.W):
                p_Xi_1[subseq] += np.log( f_j[pos][self.I[subseq][pos] - 1] )
                p_Xi_2[subseq] += np.log( f_0[self.I[subseq][pos]-1])
        
        return np.exp(p_Xi_1), np.exp(p_Xi_2)
    
    def count_letter_appearance(self): #done
        '''
        count the total appearance times for each alphabets
        Arguments:
        data: input source W-mer data, size: n x W
        return:
        count: counting results, size: n x 4
        '''

        count = np.zeros( (self.n, 4) , dtype = 'int')

        for i in range(self.n):
            C = Counter(self.X[i])
            count[i][0] = C['A']
            count[i][1] = C['T']
            count[i][2] = C['C']
            count[i][3] = C['G']

        return count
    
    def update_erasing(self):
        '''
        compute the erasing values e_ij
        '''
        
        for i in range(self.N):
            for j in range(self.l[i]):
                lower_bound = j - self.W + 1 if j - self.W + 1 >= 0 else 0
                vals = self.z[i][lower_bound:j]
                if sum(vals) > 1:
                    vals /= sum(vals)
                if any(vals > 9.99999e-1):
                    self.e_ij[i][j] *= 1e-6
                else:
                    self.e_ij[i][j] *= np.exp( sum( np.log( np.ones(len(vals)) - vals ) ) )
                
        return
    
    def update_z(self):
        '''
        update the small z values
        '''
        
        assert sum(self.l)-self.N*(self.W - 1) == self.n, 'Error: index division not equal for updating z.'
        idx = 0
        for i in range(self.N):
            for j in range(self.l[i] - self.W + 1):
                self.z[i][j] = self.Z[idx][0]
                idx += 1
        
        return                
    
    def E_step(self): #done
        '''
        calculate the Z_ij, in article's eq(4)
        also update small z depends on the results of Z

        Arguments:
        condi_dis: conditional distribution, from the defined function, size: n x 2
        lamd: probability for using models, size: 1 x 2
        return:
        Z: membership probability, size: n x 2
        '''
        
        # regulate the shape
        p = np.array([self.p_X1, self.p_X2]).transpose()
        lamb = np.array([self.lambda1, self.lambda2]).reshape(1, 2)
        assert p.shape[0] == self.n, 'dimension error for E-step prob'

        multi_results = p * np.tile(lamb, (self.n, 1))
        summation = np.sum(multi_results, axis = 1, keepdims = True)
        Z = multi_results / summation
        self.Z = Z.copy()
        self.update_z()
        
        return Z
    
    def M_step(self):
        '''
        calculate the lambda and f_ij
        
        Arguments:
        Z: membership from E-step, size: n x 2
        I: indicator function, size: n x W
        count: count the appearance time for each alphabet in every sequences, size: n x 4
        
        return:
        
        '''
        
        Z = self.Z.transpose()
        count = self.letter_counts.transpose()

        # update lambda, eq(5)
        lamb = np.mean(Z, axis = 1)
        
        # update f_ij
        # calculate c_0k and c_jk
        c_0k = np.zeros((1, self.L))
        c_jk = np.zeros((self.W, self.L))
        
        # calculate the c_0k
        for k in range(self.L):
            for i in range(self.n):
                for j in range(self.W):
                    c_0k[0][k] += Z[1][i] if self.I[i][j] == (k + 1) else 0
        
        '''for i in range(4):
            c_0k[0][i] = np.sum( Z[1] * count[i] )'''
        
        # calculate the c_jk
        # first make erasing being 1 x n
        E = np.hstack([self.e_ij[i][:(self.l[i] - self.W + 1)] for i in range(self.N)])
        EZ = E * Z[0]
        
        for pos in range(self.W):
            for i in range(self.n):
                c_jk[pos][self.I[i][pos] - 1] += EZ[i]
        
        # start updating f
        # won't directly update self.f since it will use the previous values to check the disparity
        c_0k += self.beta_i
        c_0k /= (np.sum(c_0k, axis = 1) + self.beta)
        
        for pos in range(self.W):
            c_jk[pos] += self.beta_i.squeeze(0)
            c_jk[pos] /= (np.sum(c_jk[pos]) + self.beta)
            
        f_ij = np.vstack((c_0k,c_jk))
        
        self.update_erasing()
        
        return lamb, f_ij
    
    def update_variables(self, lamb, f_ij):
        '''
        update the variables' values after M step
        it will update: f_ij, lambda, conditional distribution p_X1 and p_X2, erasing
        
        Inputs:
        lamb: lambda, size: 1 x 2
        f_ij: the updated values for position-wise letter frequencies, size: (W+1) x L
        '''
        
        self.lambda1 = lamb[0]
        self.lambda2 = lamb[1]
        self.f_ij = f_ij.copy()
        self.p_X1, self.p_X2 = self.condi_distribution(self.f_ij)
        self.update_erasing()
        
        return
    
    def iter(self, epsilon = 1e-6):
        '''
        iterate the E and M steps
        '''
        
        err_ls = list()
        f_ls = list()
        f_ls.append(self.f_ij)
        
        for step in range(1000):
            condi_dis = np.array([self.p_X1, self.p_X2])
            lamb = np.array([self.lambda1, self.lambda2])
            new_Z = self.E_step()
            new_lambda, new_f_ij = self.M_step()
            f_ls.append(new_f_ij)
            
            err = np.sqrt(np.sum( (self.f_ij - new_f_ij) ** 2  , axis = 1))
            err_ls.append(err)
            
            if all(err < epsilon):
                #print('At step {}, EM converges to a solution.'.format(step))
                #print('err is ', err)
                self.update_variables(new_lambda, new_f_ij)
                break
            
            '''if step % 100 == 0 and step > 0:
                print('in step {} err is {}'.format(step, err))
            '''
            self.update_variables(new_lambda, new_f_ij)
        
        
        
        return #err_ls, new_f_ij, f_ls
    
    def show_seq(self):
        '''
        select the motif
        '''
        
        #spec = np.log(self.f_ij[1:]/self.f_ij[0])
        t = np.log((1-self.lambda1)/self.lambda1)
        s = np.log(self.p_X1/self.p_X2)
        motifs = list( compress(self.X, s > t) )
        
        return motifs
    
    def prob_out(self):
        '''
        output the last probability results for motifs
        '''
        
        motifs = self.show_seq()
        pos_prob = np.zeros((self.W, self.L))
        logo_tmp = [list() for _ in range(self.W)]
        
        for seq in motifs:
            for i in range(len(seq)):
                pos_prob[i][ alphabet_dict[seq[i]] - 1 ] += 1
                if seq[i] not in logo_tmp[i]:
                    logo_tmp[i].append(seq[i])
        
        pos_prob /= len(motifs)
        
        # sort the logo
        for i in range(len(logo_tmp)):
            if len(logo_tmp[i]) > 1:
                sorted_logo = sorted(list(map(lambda x: alphabet_dict[x], logo_tmp[i])))
                res = list(map(lambda x: val_to_char[x], sorted_logo))
                logo_tmp[i] = res

        logo = list()
        for l in logo_tmp:
            logo.append(''.join(l))
        
        return pos_prob, logo
    

In [59]:
def preprocess_seq(input_seq, IO, pad_num = 17):
    '''
    adjust the length for single sequence
    tokenize the sequence
    '''
    
    L = len(input_seq)
    
    if IO:
        if L > 498:
            mid = L//2
            seq = input_seq[mid - 249: mid + 249]
            res = np.array(list(map(lambda x: combi_letters[x], seq)))
            res = np.concatenate((np.array([0]), res, np.array[16]))
        else:
            res = np.array(list(map(lambda x: combi_letters[x], input_seq)))
            res = np.concatenate((np.array([0]), res, np.array([16]), np.array([pad_num] * (498 - L) )))
        assert len(res) == 500, 'Preprocess the input sequence not equal to 500'
    
    else:
        assert L <= 48, 'Preprocess the output sequence get the length error, > 48'
        res = np.array(list(map(lambda x: combi_letters[x], input_seq)))
        res = np.concatenate((np.array([0]), res, np.array([16]), np.array([pad_num] * (48 - L))))
    
    return res.astype('int')

def make_outputs(data, row_size):
    '''
    make the outputs for the model, i.e. make the tgt
    return:
    tgt: the target array, has transfered the string data into tokens
    '''
    tgt = list()
    l = list()
    for seq in data:
        l.append(len(seq))
    assert min(l) > 3, 'The minimum length of the inputs are less than 3'
    if min(l) < row_size:
        row_size = min(l)
        print("Mention: the target size is greater than the minimum length of the data, it will set the size as the minimum length of the data")
    
    for motif_len in range(row_size - 3):
        #print('At step ', motif_len)
        m = MEME(data, motif_len + 3)
        m.iter()
        motif, logo = m.prob_out()
        tgt.append(preprocess_seq(logo, 0))
    tgt = np.array(tgt)
    assert len(tgt.shape) == 2, 'tgt dimension error in make outputs'
    return tgt

def expand_dim(data, IO, pad_num = 17, row_size = 40):
    '''
    To expand input and output data into specific dimensions
    The input variable of this function is transfered matrix, that is the matrix with tokenized elements
    return:
    res: the results of expanding the dimension
    '''

    row, col = data.shape
    assert row < row_size, 'Row size out of range.'

    if IO:# expand input to row_size x 500
        assert col <= 500, 'Column size out of range(Input)'
        col_expand = pad_num * np.ones((row, 500 - col))
        row_expand = pad_num * np.ones((row_size - row, 500))

    elif not IO:# expand output to row_size x 50
        assert col <= 50, 'Column size out of range(Output)'
        col_expand = pad_num * np.ones((row, 50 - col))
        row_expand = pad_num * np.ones((row_size - row, 50))

    res = np.concatenate( (np.concatenate((data, col_expand), axis = 1), row_expand), axis = 0)

    return res


def get_padding_mask(self, data, pad_num = 7):
    mask = torch.zeros(data.size())
    mask[data == pad_num] = -torch.inf
    return mask

In [61]:
data = ['ATCGGGTTC','ATTTGCGTC']
tgt = make_outputs(data, 50)
ex = expand_dim(tgt,0)

Mention: the target size is greater than the minimum length of the data, it will set the size as the minimum length of the data


In [65]:
mdl = TransformerModel(d_model, dropout, max_len, nhead, encoder_layer_nums, decoder_layer_nums, dim_ff, ntoken)

In [ ]:
class trainer(object):
    def __init__(self, model, data_in, W, row_size = 40):
        
        self.model = model
        self.data = data_in
        self.row_size = row_size
        self.W = W
    
    


    
    def train(self):
        #tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt.size(-1))
            

In [ ]:
if __name__ == '__main__':
    
    path = ''
    mdl = TransformerModel()
    
    # read data
    data = read_file(path)
    # build model and train
    
    T = trainer()
    T.train()

In [ ]:
data = ['CAGTCAGCAC','AGTTACGTAG']
data1 = ['AAAAA','AAAAA']
m = MEME(data, 3)

In [ ]:
mdl = TransformerModel(d_model, dropout, max_len, nhead,
                       encoder_layer_nums, decoder_layer_nums,
                       dim_ff, ntoken)

In [ ]:
src = torch.LongTensor([
    [0,3,2,4,3,3,2,1,5,6,6],
    [0,4,3,1,2,3,1,2,1,5,6]
])

tgt = torch.LongTensor([
    [0,3,2,4,3,3,2,1,5,6],
    [0,4,3,1,2,3,1,2,1,5]    
])

mdl(src,tgt).shape

In [ ]:
emb = mdl.embedding
mdl.decoder(emb(tgt),emb(src)).shape